# @chain 데코레이터를 사용하여 Runnable 객체 생성

`@chain` 데코레이터를 추가하여 임의의 함수를 체인으로 변환할 수 있습니다.

이는 함수를 [`RunnableLambda`](./functions)로 래핑하는 것과 기능적으로 동일합니다.

실제 동작을 살펴보겠습니다!


In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH13-LCEL-Advanced")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH13-LCEL-Advanced


In [3]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import chain
from langchain_openai import ChatOpenAI

`ChatPromptTemplate` 클래스를 사용하여 두 개의 프롬프트 템플릿을 정의합니다.

- `prompt1` 은 주어진 주제에 대한 짧은 설명을, <br>`prompt2` 는 영어로 번역해 달라는 요청 프롬프트 입니다.


In [4]:
# 프롬프트 템플릿을 정의합니다.

# 먼저 실행되어야 함
prompt1 = ChatPromptTemplate.from_template("{topic} 에 대해 짧게 한글로 설명해주세요.")

# 그 후에 실행되어야 함
prompt2 = ChatPromptTemplate.from_template(
    "{sentence} 를 emoji를 활용한 인스타그램 게시글로 만들어주세요."
)

# 어떻게 체인을 연결지을 것인가?

`custom_chain` 함수는 입력 텍스트를 기반으로 사용자 정의 체인을 실행합니다.

- `@chain` 데코레이터로 사용자 정의 함수를 데코레이팅 하며, 데코레이팅을 통해 함수를 `Runnable` 한 객체로 만듭니다.


In [5]:
@chain
def custom_chain(text):
    # 첫 번째 프롬프트, ChatOpenAI, 문자열 출력 파서를 연결하여 체인을 생성합니다.
    chain1 = prompt1 | ChatOpenAI(model="gpt-4o-mini") | StrOutputParser()
    output1 = chain1.invoke({"topic": text})

    # 두 번째 프롬프트, ChatOpenAI, 문자열 출력 파서를 연결하여 체인을 생성합니다.
    chain2 = prompt2 | ChatOpenAI(model="gpt-4o-mini") | StrOutputParser()
    # 두 번째 체인을 호출하여 파싱된 첫 번째 결과를 전달하고 최종 결과를 반환합니다.
    return chain2.invoke({"sentence": output1})

custom_chain은 이제 실행 가능한 객체(`runnable`)이므로, `invoke()` 를 사용하여 실행해야 합니다.

- LangSmith 추적 기록을 확인해 보면, `custom_chain` 추적 정보가 있을 것이며, 그 아래에는 OpenAI 호출이 중첩되어 있을 것입니다.
- [LangSmith 추적 링크](https://smith.langchain.com/public/17023c06-e53a-4711-a42a-2fbc51249883/r)


In [6]:
# custom_chain을 호출
print(custom_chain.invoke("양자역학"))

🌌✨ 양자역학의 세계에 오신 것을 환영합니다! 🔬💫

양자역학은 원자와 아원자 입자의 신비로운 세계를 탐구하는 물리학의 한 분야입니다. 🌌🔍 고전물리학으로 설명할 수 없는 현상들을 다루며, 입자의 파동성 🌊, 불확정성 원리 ❓, 양자 얽힘 🔗 등의 흥미로운 개념이 가득합니다!

예를 들어, 불확정성 원리는 우리가 입자의 위치와 운동량을 동시에 정확히 알 수 없다는 것을 의미해요! 🤯💥 이 놀라운 원리는 양자역학의 핵심 중 하나랍니다.

양자역학은 현대 물리학의 기초를 이루며, 반도체 💻, 레이저 🔦, 양자 컴퓨팅 💡 등 다양한 기술의 발전에 기여하고 있어요!

🔭💖 양자세계의 매력을 함께 느껴보아요! #양자역학 #물리학 #과학의미래 #신비로운세계 #기술혁신
